# Sparseを実装する

本名 sparse vector algorithm. Dwork and Roth 2014を参照。

PRESLにおいてはdifferential privateにLPを解くために利用される。

気持ちとしては下のような感じ。

離散化した値の組合せそれぞれに対して、LPの解を計算したい。が、その出力には差分プライバシーを守っていてほしい。
ので、LPをそれぞれに対して解いた時に、その解を貯めておき、それをSparseにぶっこむことで出力にノイズを入れながら、その解の中で重要なものは一定の確率で得られているようにする。


なのでSparse自体はめっちゃ簡単。単なる数列に対してノイズを加え、出力するためのthresholdにもノイズを加えながら一定以上の値あは出力できるようにするだけ。


試しにデータを引き出すタスクをやってみる（このタスクが問題設定に対して適切じゃなさそうな…）
実装するのは楽という確認ぐらいの気持ちで

In [68]:
import numpy as np
from numpy.random import laplace
np.random.seed(123)

In [122]:

datasize = 10000
T = 0
num_answers = 500
epsilon = 100
# data がこの時gamma = 2
data = np.random.uniform(low = -1, high = 1, size = datasize)
gamma = 2

In [123]:
output = np.zeros(datasize)

def sparse(data, T, num_answers, epsilon):
    T_hat = T + laplace(scale = 2*gamma/epsilon)
    sigma = 2*num_answers*gamma/epsilon
    count = 0
    count2 = 0
    for i in range(datasize):
        nu = laplace(scale = sigma)
        Q_hat = data[i] + nu
        count2 += 1
        if Q_hat <= T_hat:
            output[i] = Q_hat
            count += 1
            
            if count >= num_answers:
                break
        
        else:
            output[i] = np.nan
    
    print(count)
    print(count2)
    return output[:count2],count2

In [124]:
result, num_query = sparse(data, T, num_answers, epsilon)

500
1001


In [125]:
dataslice = data[:len(result)]

In [126]:
diff = dataslice - result 

In [127]:
beta = 0.99
alpha = 4*num_answers*gamma *(np.log(num_query) + np.log(2*num_answers/beta))/epsilon

In [128]:
alpha

553.0624157660344

In [129]:
# タスクがあかんっぽいな
sum([1 for i in diff if np.abs(i) > alpha ])

0